# Предсказание непрерывной переменной по тексту: сумма сбора 

Задача: на материале датасета, состоящего из описаний благотворительных сборов, проверить, может ли модель по тексту предсказывать сумму, которую соберет проект. Можно рассматривать это как проверку наличия зависимости между текстами и успешностью сбора, выраженной через скорость сбора.  

In [195]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge, LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, r2_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd
import pymorphy2
morph_analyzer = pymorphy2.MorphAnalyzer()
from nltk.corpus import stopwords
from string import digits
import re
import numpy as np
from nltk import word_tokenize
from scipy.sparse import csr_matrix
from collections import Counter

In [2]:
# pip install gensim

In [6]:
import gensim

### Импорт данных

In [8]:
del df # очистим переменную

NameError: name 'df' is not defined

In [9]:
df = pd.read_csv('/Users/liza/PycharmProjects/Planeta_project/plset_ver_010.csv')

In [10]:
df.head()

,Unnamed: 0,Title,Author,Description,Briefly,Goal,Result,Donations,Start,Finish,...,Started,Finished,LeadBag,DaysLong,Cluster,Category,Rate,Names,Adr,NamesSimRate
0,0,Подарим жизнь,Бобруйское общественное объединение защиты жив...,"Друзья, рады приветствовать на странице нашего...",Спасение животных от эвтаназии на 5 сутки посл...,10000.0,76100,67,11 сентября 2017,9 ноября 2017,...,11 Sep 2017,9 Nov 2017,"['спасение', 'животное', 'эвтаназия', 'сутки',...",59,7,животные,1290.0,['Эгиды'],False,0
1,1,Оплата нянечек для отказных детей в больницах,"Благотворительный фонд ""Дети без мам""","Когда ребёнок серьёзно болен, он ложится в бол...",Сбор средств на оплату ухода за детьми-сиротам...,150000.0,711665,470,1 июля 2014,16 сентября 2014,...,1 Jul 2014,16 Sep 2014,"['сбор', 'средство', 'оплата', 'уход', 'ребёно...",77,0,сироты_дети_из_неблагополучных семей,9240.0,[],False,0
2,2,Оплата нянь для детей-отказников в больницах,"Благотворительный фонд ""Дети без мам""","В больницах живут малыши, оставшиеся без попеч...",Сбор средств на уход за детьми-отказниками в б...,50000.0,236257,202,14 октября 2013,16 января 2014,...,14 Oct 2013,16 Jan 2014,"['сбор', 'средство', 'уход', 'ребёнок', 'отказ...",94,0,сироты_дети_из_неблагополучных семей,2510.0,"['Зою', 'Ксану', 'Анну', 'Юрия', 'Фёдора', 'Ал...",False,11
3,3,"""Не вешай нос! Или право на жизнь""",Местная общественная организация защитников жи...,"И снова здравствуйте, Друзья! \nМеня зовут Над...",Цель проекта - ликвидировать долги за ветерина...,29395.0,95300,170,21 февраля 2017,21 апреля 2017,...,21 Feb 2017,21 Apr 2017,"['цель', 'проект', 'ликвидировать', 'долг', 'в...",59,7,животные,1620.0,"['Надежда', 'Кипиш', 'Островка', 'Маня', 'Хрюн...",False,35
4,4,Календарь реабилитационного центра для птиц,Вера Пахомова,"Привет, друг! Мы снова на Planeta.ru!) \n В н...","""Воронье Гнездо"" собирает средства на печать к...",15000.0,42550,87,9 ноября 2019,8 декабря 2019,...,9 Nov 2019,8 Dec 2019,"['воронья', 'гнездо', 'собирать', 'средство', ...",29,4,активизм_просвещение_профилактика,1470.0,[],False,0


In [321]:
# оказалось, что при переформатировании дат начала и конца сбора старые колонки (с русскими названиями месяцев) не удалились, удалим их 

df = df.drop(['Start', 'Finish'], axis=1) # “axis 0” represents rows and “axis 1” represents columns.
df = df.to_csv("plset_ver_011.csv") # сохраняем датасет
df = pd.read_csv('/Users/liza/PycharmProjects/Planeta_project/plset_ver_011.csv')# читаем снова

⬆︎ Примечание к drop. When inplace = True, the data is modified in place, which means it will return nothing and the dataframe is now updated. When inplace = False, which is the default, then the operation is performed and it returns a copy of the object. 

In [12]:
df.dtypes # проверим, какие типы данных содержит датасет

Unnamed: 0        int64
Unnamed: 0.1      int64
Title            object
Author           object
Description      object
Briefly          object
Goal            float64
Result            int64
Donations         int64
Start            object
Finish           object
LengthZn          int64
LengthWd          int64
NumSent           int64
NumSentNLTK       int64
Started          object
Finished         object
LeadBag          object
DaysLong          int64
Cluster           int64
Category         object
Rate            float64
Names            object
Adr                bool
NamesSimRate      int64
dtype: object

\
Дальше нас будет интересовать переменная (колонка) Rate. Проверим ее. 

In [13]:
Counter(pd.isna(df['Rate'] == True)) # есть ли пустые значения (--> нет)

Counter({False: 2038})

In [14]:
Counter(np.isfinite(df['Rate'])) # все ли значения - это конечные числа? (--> увы, нет)

Counter({True: 2031, False: 7})

In [15]:
df[np.isfinite(df['Rate'])==False] # смотрим, что это за бесконечные значения

,Unnamed: 0,Unnamed: 0.1,Title,Author,Description,Briefly,Goal,Result,Donations,Start,...,Started,Finished,LeadBag,DaysLong,Cluster,Category,Rate,Names,Adr,NamesSimRate
58,58,58,Никита Кольцов: продолжаем бороться,"Благотворительный фонд ""Золотце""","Благотворительный фонд ""Золотце"" создан 7 дека...",Диагноз: перенесенная гипоксия тяжелой формы с...,25000.0,31094,46,20 января,...,20 Jan,4 Feb 2020,"['диагноз', 'перенести', 'гипоксия', 'тяжёлый'...",0,0,дети_лечение_реабилитация,inf,"['Кольцова', 'Никиты', 'Никиты', 'Никита']",True,61
97,97,97,Безнадежности - нет!,АНО «ЦСИ «Русский Дом»,Психическое заболевание для человека - это ког...,Проект направлен на поддержку благотворительно...,500000.0,560685,124,31 июля 2018,...,31 Jul 2018,29 Feb 2019,"['проект', 'направить', 'поддержка', 'благотво...",0,4,социализация_возможности,inf,"['Цветаевой', 'Шиннед', 'Шиннед', 'Андрею', 'М...",False,40
453,453,453,"Вольер-спасатель для приюта ""Тимошка""",galina.gershkovich,"Мы – приют ""Тимошка. Скорая помощь""!\nНаш прию...",Как оказать экстренную помощь искалеченному жи...,84720.0,85320,153,22 октября 2019,...,22 Oct 2019,20 Feb,"['оказать', 'экстренный', 'помощь', 'искалечит...",0,7,животные,inf,[],False,0
488,488,488,Лежачему инвалиду необходима специализированна...,"Благотворительный Фонд ""Гольфстрим""","Дорогие наши!\n Благотворительный фонд ""Гольф...","Просим Bас, помочь улучшить качество жизни леж...",45000.0,45100,2,26 апреля 2014,...,26 Apr 2014,26 Apr 2014,"['просить', 'ас', 'помочь', 'улучшить', 'качес...",0,11,взрослые_лечение_реабилитация,inf,"['Казаков', 'Саше', 'Сашу', 'Сашу', 'Сашина', ...",True,58
701,701,701,Помощь детскому хоспису,Благотворительный фонд «Дом с маяком»,"Когда Мурад только появился на свет, родители ...","Для того, чтобы Мурад чувствовал себя хорошо и...",567336.0,567372,906,19 июня 2019,...,19 Jun 2019,20 Feb,"['мурад', 'чувствовать', 'оставаться', 'дом', ...",0,11,дети_лечение_реабилитация,inf,"['Мурад', 'Мурад', 'Мурадом', 'Мурада', 'Гиршп...",True,77
1286,1286,1286,УЛИЦА СЕЗАМ,"""Центр по предоставлению государственных услуг...",Наша команда - Центр социальной защиты населен...,приобретение ростовых кукол и кукол-перчаток д...,65000.0,8987,26,6 сентября 2019,...,6 Sep 2019,20 Mar,"['приобретение', 'ростовый', 'кукла', 'кукла',...",0,10,социализация_возможности,inf,"['Зелибобу', 'Бусинку', 'Кубика']",False,36
1396,1396,1396,Разбудить Ваню,Некоммерческое партнерство Международный благо...,Благотворительный центр «Надежда» официально ...,Ване 8 лет. Диагноз: Ранний детский аутизм - п...,107000.0,10202,41,13 июня 2019,...,13 Jun 2019,20 Mar,"['ван', 'год', 'диагноз', 'ранний', 'детский',...",0,1,дети_лечение_реабилитация,inf,"['Вани', 'Ваню', 'Ваня', 'Вани', 'Ваню', 'Ваня...",True,76


Данные в колонке Rate - это суммв сбора, поделенная на продолжительность сбора. Иногда продолжительность формально = 0 дней (причины: ошибки данных на площадке или при сборе датасета). При делении на 0 получились значения inf. Их невозможно обработать при обучении, поэтому от них надо избавиться или заменить inf средними/медианными значениями. К счастью, у нас таких всего 7, не так много, удалим их. ⬇︎

In [16]:
df = df.replace([np.inf, -np.inf], np.nan) # сначала меняем inf на nan

In [17]:
Counter(pd.isna(df['Rate'] == True)) # почему-то количество na не изменилось, все 2038 строк как бы не na (видимо, превращенные в na читаются как-то иначе)

Counter({False: 2038})

In [18]:
df = df.dropna(subset = ["Rate"]) # удаляем na

In [19]:
df.shape # хотя все na были false, все же они нашлись и удалились - ровно те 7 штук, которые появились из inf 

(2031, 25)

In [20]:
Counter(np.isfinite(df['Rate'])) # снова проверим, все ли значения в колонке - конечные числа (--> теперь да)

Counter({True: 2031})

In [21]:
for i in df.Rate: # все значения float (впрочем, при наличии inf они тоже были float)
    print(type(i))
    break

<class 'float'>


\
В датасете есть выбросы, избавимся от них. Все что ниже\выше 1 и 99 квантилей, выбросим. ⬇︎ (это решение пришло просле проверки предсказаний на изначальном датасете с выбросами, избавление от них - улучшает результат). 

In [186]:
# keep values within interval
df_norm = df[(df.Rate > df.Rate.quantile(0.01)) & (df.Rate < df.Rate.quantile(0.99))]

In [187]:
df_norm.shape # выбросы ушли

(1955, 25)

In [22]:
# стоп-слова

stop_words = stopwords.words('russian')
stop_words.extend(['это', '–', '-', 'фонд', 'наш', 'помощь', 'помогать',
                   'помочь', 'поддержать', 'поддержка', 'средство', 'который', 'весь',
                   'благотворительный', 'пожертвовать', 'пожертвование', 'деньги', 'рубль', 'год', 'день', 'тысяча',
                   'ваш', 'сегодня', 'завтра', 'этот', 'дать', 'проект', 'свой' ])

In [23]:
# функция для предобработки текстов

def prep(text):
    clean_text = text.translate(str.maketrans('', '', '!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~«»№!—'))
    clean_text = clean_text.translate(str.maketrans('', '', digits)) # удаляем цифры
    clean_text = re.sub("-", " ", clean_text) # меняем тире на пробелы, т.к. maketrans их почему-то не выхватывает
    #clean_text = re.sub("[a-zA-Z]", "", clean_text)  # исключаем слова латиницей
    clean_text = clean_text.lower() # приводим все к нижнему регистру
    clean_text = clean_text.split() # разбиваем (токенизируем) по словам
    
    # можно возвращать слова без лемматизации\стемминга
    #words = [word for word in clean_text if word not in stop_words]
    #return words
    
    lemmas = [morph_analyzer.parse(word)[0].normal_form for word in clean_text]
    lemmas = [word for word in lemmas if word not in stop_words]
    return lemmas
    
    #ниже - альтернатива лемматицации, можно их переключать (за/раскомменчивать)
    #stemmer = RussianStemmer()
    #stemmed_words = [stemmer.stem(word) for word in clean_text]
    #return stemmed_words

### Векторизация

Если векторизовать все тексты описаний из датасета и проверить объем (bag_of_words.shape), то мы увидим вокабуляр всего датасета. Это более 30.000 слов (лемм), то есть получается более 30.000 признаков, по которым модель должна предсказать сумму. Это очень много, результаты модели в таком случае - с отрицательным score. Поэтому в векторизатор добавим параметр max_features, то есть отставим только топовые признаки. В результате перебора вручную значений 10, 15, 20, 30, 50 и 100 - лучший результат у 30. TfiDf и Count векторизаторы показывают примерно одинаковый результат. Сокращение количества признаков положительно влияет на оба.

In [307]:
%%time

vec = TfidfVectorizer(tokenizer=prep, max_features = 30) # сократим длину вектора, иначе слишком много признаков
bag_of_words = vec.fit_transform(df_norm.Description)

CPU times: user 3min 15s, sys: 924 ms, total: 3min 16s
Wall time: 3min 21s


In [319]:
bag_of_words.shape # вокабуляр всего датасета более 30.000 слов (признаков), мы оставили 30 признаков

(1955, 30)

### Обучение и оценка
Для предсказания непрерывной переменной используются линейные модели. [В sklearn они описаны тут.](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model) 

In [142]:
X_train, X_test, y_train, y_test = train_test_split(bag_of_words, df_norm.Rate, random_state=42) 

In [144]:
linreg = LinearRegression().fit(X_train, y_train) # вызов и обучение
linreg.predict(X_test) # предсказание, # если числа очень дробные, jupyter выведет Scientific notation (экспоненциальная запись), дальше можно округлить
linreg.score(X_test,y_test) # оценка

0.017677628857102445

In [320]:
# напечатаем таблицу сравненения предсказанных сумм и реальных

compar = pd.DataFrame.from_dict({'predicted':linreg.predict(X_test), 'fact':y_test})
compar['difference'] = compar.predicted - compar.fact
compar.sample(n=10).round(2) # здесь можно регулировать округление и количество выводимых строк

,predicted,fact,difference
958,2605.32,1720.0,885.32
1772,2190.51,30.0,2160.51
1734,1695.33,100.0,1595.33
1270,1712.48,1240.0,472.48
1839,2624.10,60.0,2564.10
442,3051.03,780.0,2271.03
618,1428.83,760.0,668.83
259,2852.13,13520.0,-10667.87
1632,2489.21,180.0,2309.21
375,2702.06,240.0,2462.06


Сравним еще несколько моделей, линейных и один из ансабмлей (RandomForestRegressor). The goal of ensemble methods is to combine the predictions of several base estimators built with a given learning algorithm in order to improve generalizability/robustness over a single estimator. ⬇︎

In [148]:
ridge = Ridge().fit(X_train, y_train)
ridge.predict(X_test) # если числа очень дробные, jupyter выведет Scientific notation (экспоненциальная запись), дальше можно округлить
print(ridge.score(X_test,y_test))

0.01757492131245897


In [149]:
lasso = Lasso().fit(X_train, y_train)
lasso.predict(X_test)
print(lasso.score(X_test,y_test))

0.017236783894583052


In [150]:
forest = RandomForestRegressor().fit(X_train, y_train) # вызов модели и обучение
forest.predict(X_test) # предсказание
print(forest.score(X_test,y_test)) # оценка

compar_for = pd.DataFrame.from_dict({'predicted':forest.predict(X_test), 'fact':y_test})
compar_for['difference'] = compar_for.predicted - compar_for.fact
compar_for.sample(n=10).round(2)

-0.03423455612716153


,predicted,fact,difference
586,2492.30,1310.0,1182.30
190,4232.10,20180.0,-15947.90
1158,3742.10,200.0,3542.10
656,1869.50,3920.0,-2050.50
1717,710.70,40.0,670.70
832,680.70,1040.0,-359.30
1861,2542.10,80.0,2462.10
572,1330.87,4070.0,-2739.13
440,3408.70,3660.0,-251.30
647,2776.30,1870.0,906.30


Комментарий к оценке. Метод .score показывает коэффициент детерминации (R^2 — R-квадрат). Его рассматривают как универсальную меру зависимости одной случайной величины от множества других. The coefficient R^2 is defined as (1 - u/v), where u is the residual sum of squares ((y_true - y_pred) ** 2).sum() and v is the total sum of squares ((y_true - y_true.mean()) ** 2).sum(). The best possible score is 1.0 and it can be negative (because the model can be arbitrarily worse). A constant model that always predicts the expected value of y, disregarding the input features, would get a R^2 score of 0.0.

Итог. Результаты низкие, максимальный, которого удалось достичь с разными моделями и размерностью векторов: 0.0176 (Логрег). При этом все линейные модели показывают почти одинаковый результат. У ансамблей он намного хуже. 

Попробуем теперь разбить Rate на ранги. 

In [222]:
# Посмотрим, как распределены данные с точки зрения скорости сбора (вывод в рублях)

print(df_norm.Rate.quantile(0.25))
print(df_norm.Rate.quantile(0.50))
print(df_norm.Rate.quantile(0.75))

190.0
720.0
2245.0


In [309]:
# функция для категоризации данных

def df_ranking(df):
    bins = [-np.inf, 600, 3000, np.inf] # эти значения будут границами разметки 
    labels = ['slow','moderate', 'fast'] # какие теги присвоить категориям 
    df['RateRank'] = pd.cut(df['Rate'], bins=bins, labels=labels) # pandas.cut allows to segment and sort data values into bins, пишем новую колонку с рангами
    return(df)

In [310]:
del df_ranked # для очистки переменной

In [311]:
df_ranked = df_ranking(df_norm.copy()) # без .copy датасет, который подается в функцию, тоже будет изменен, а я хочу его сохранить для сравнения

In [312]:
Xx_train, Xx_test, yy_train, yy_test = train_test_split(bag_of_words, df_ranked.RateRank, random_state=42) # повторим разбиение с новыми переменными, чтобы не стирать старые

In [313]:
logreg = LogisticRegression(max_iter=5000).fit(Xx_train, yy_train) # обучаем

In [314]:
print(classification_report(yy_test, logreg.predict(Xx_test))) # тестируем

              precision    recall  f1-score   support

        fast       0.41      0.09      0.15        96
    moderate       0.45      0.35      0.40       171
        slow       0.53      0.80      0.64       222

    accuracy                           0.50       489
   macro avg       0.46      0.41      0.39       489
weighted avg       0.48      0.50      0.46       489



⬆︎ При изменении границ категорий и их количества (3 или 4 ранга) оценки довольно существенно меняются. Если разбить четко по квартилям - низкие и консистентные, все в районе 30%, то есть, видимо, модель просто делит данные пропорционально. Это было обучение на векторах с max_features=30. Но с рангами можно попробовать вернуться к большим векторам.⬇︎ 

In [315]:
%%time

vec_full = TfidfVectorizer(tokenizer=prep) # убрали max_features
bag_of_words_full = vec_full.fit_transform(df_ranked.Description)

CPU times: user 3min 10s, sys: 662 ms, total: 3min 11s
Wall time: 3min 13s


In [316]:
Xxx_train, Xxx_test, yyy_train, yyy_test = train_test_split(bag_of_words_full, df_ranked.RateRank, random_state=42)
logreg = LogisticRegression(max_iter=5000).fit(Xxx_train, yyy_train) # обучаем
print(classification_report(yyy_test, logreg.predict(Xxx_test))) # тестируем

              precision    recall  f1-score   support

        fast       0.70      0.17      0.27        96
    moderate       0.48      0.42      0.45       171
        slow       0.58      0.82      0.68       222

    accuracy                           0.55       489
   macro avg       0.59      0.47      0.47       489
weighted avg       0.57      0.55      0.52       489



С полными векторами результат лучше! 

## Общий вывод. 
Механизм работает, и теоретически на практике такую модель можно было бы использовать для оценки примерного потенциала написанного текста. Хотя пока качество предсказания очень низкое, но есть ощушение, что оно не бессмысленное и его можно повышать: пробуя другие модели, настраивая гиперпараметры, меняя размерность векторов, меняя способ векторизации текстов.\
В частности, векторизовать тексты можно с помощью word2vec (gensim, rusvectores или обучать свою, допустим, doc2vec). Но нужно разобраться с тем, как зафиттить векторы на свои данные (метода fit_transform, как у CounVectorizer и TfidfVectorizer, у них нет) и получить на выходе scipy.sparse.csr.csr_matrix (как bag_of_words = vec.fit_transform(df.Description)) для подачи в train_test_split. При этом, видимо, для предсказания непрерывной переменной, то есть суммы, а не ранга, понадобится сокращение размерности вектора, так как практика показала что даже 50 признаков - слишком много для моделей, а минимальная размерность предобученных моделей word2vec (rusvectores) - 100. 